In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv('./smartphone_cleaned_v5.csv')

In [27]:
df.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,6.70,120,1440 x 3216,3,1.0,android,50.0,16.0,0,NaN
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,6.59,120,1080 x 2412,3,1.0,android,64.0,16.0,1,1024.0
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,6.60,90,1080 x 2408,3,1.0,android,50.0,13.0,1,1024.0
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,6.55,120,1080 x 2400,3,1.0,android,50.0,16.0,1,1024.0
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,6.70,120,1080 x 2412,3,1.0,android,108.0,16.0,0,NaN


In [28]:
objects = df[df["rating"].isnull()]
objects

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
14,samsung,Samsung Galaxy S23 Ultra 5G,114990,NaN,True,True,False,snapdragon,8.0,3.20,...,6.80,120,1440 x 3088,4,1.0,android,200.0,12.0,0,NaN
29,oneplus,OnePlus 11 Pro,69999,NaN,True,True,False,snapdragon,8.0,3.20,...,6.70,120,1440 x 3216,3,1.0,android,50.0,16.0,0,NaN
37,samsung,Samsung Galaxy S22 Ultra 5G,91999,NaN,True,True,False,snapdragon,8.0,3.00,...,6.80,120,1440 x 3080,4,1.0,android,108.0,40.0,0,NaN
49,samsung,Samsung Galaxy A74 5G,42999,NaN,True,True,False,snapdragon,8.0,2.36,...,6.70,120,1080 x 2412,4,1.0,android,108.0,32.0,1,1024.0
69,oppo,Oppo Find N Fold,99990,NaN,True,True,False,snapdragon,8.0,3.20,...,7.10,120,1792 x 1920,2,NaN,other,50.0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,huawei,Huawei Mate X,169000,NaN,True,True,True,kirin,8.0,2.86,...,8.00,60,2200 x 2480,4,NaN,NaN,48.0,NaN,0,NaN
957,vivo,Vivo Y55S,13490,NaN,False,False,False,snapdragon,4.0,1.20,...,5.20,60,720 x 1280,1,1.0,android,13.0,5.0,1,NaN
963,lava,Lava X3,6999,NaN,False,False,False,helio,4.0,2.00,...,6.50,60,1600 x 720,2,1.0,android,8.0,5.0,1,512.0
972,itel,itel A23s,4787,NaN,False,False,False,spreadtrum,4.0,1.40,...,5.00,60,854 x 480,1,NaN,android,2.0,NaN,0,NaN


In [47]:
def evaluate_resolution(expression):
    try:
        # Replace special characters and evaluate the expression
        expression = expression.replace(" ", "").replace("x", "*")
        result = eval(expression)
        return 10 if result > 2590000 else 5
    except Exception as e:
        # Handle any errors that occur during evaluation
        print(f"Error evaluating expression '{expression}': {e}")
        return False
    
def categorize_phone(row):
    condition_fast_processor = row["processor_speed"]*10/max_speed
    condition_fluid_display = 10 if row["refresh_rate"] >60 else 5
    condition_many_cores = row["num_cores"]*10/max_cores
    condition_nfc = 5 if row["has_nfc"]==True  else 0
    condition_5g=5 if row["has_5g"]==True else 0
    condition_resolution = evaluate_resolution(row["resolution"])
    total = condition_5g+condition_fluid_display +  condition_fast_processor + condition_nfc + condition_many_cores +condition_resolution
    if total > 40: return "flagship"
    elif total >30: return "midrange"
    else: return "average"
    # no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
    
def categorize_camera(row):
    c1 = 10 if row["phone_category"] == "flagship" else 5
    c2 = 10 if row["primary_camera_rear"] > 12 else 5
    c3 = 5 if row["brand_name"] in ['apple', 'samsung'] else 0
    total = c1+c2+c3
    if total == 25: return "high"
    elif total >20: return "midrange"
    else: return "low"
    # no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
    

In [ ]:
# condition_fast_processor = df["processor_speed"]>3
# condition_fluid_display = df["refresh_rate"]>60
# condition_many_cores = df["num_cores"]>=8
# condition_nfc = df["has_nfc"]==True
# condition_5g=df["has_5g"]==True
# condition_resolution = df["resolution"].apply(evaluate_resolution)
# no_of_conditions_satisfied = condition_5g.astype(int)+condition_fluid_display.astype(int) +  condition_fast_processor.astype(int) + condition_nfc.astype(int) + condition_many_cores.astype(int) +condition_resolution.astype(int)
# # all_conditions = condition_fluid_display &  condition_fast_processor & condition_nfc & condition_5g & condition_many_cores & condition_resolution
# # most_conditions = 
# # average_conditions = 
# # better_than_no_conditions = 
# # no_conditions = 
# conditions = no_of_conditions_satisfied == 6

In [41]:
from copy import deepcopy
df2 = deepcopy(df)
df2.head()
final_rec = deepcopy(df)
final_rec.head()


,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,6.70,120,1440 x 3216,3,1.0,android,50.0,16.0,0,NaN
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,6.59,120,1080 x 2412,3,1.0,android,64.0,16.0,1,1024.0
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,6.60,90,1080 x 2408,3,1.0,android,50.0,13.0,1,1024.0
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,6.55,120,1080 x 2400,3,1.0,android,50.0,16.0,1,1024.0
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,6.70,120,1080 x 2412,3,1.0,android,108.0,16.0,0,NaN


In [50]:
objects = df[df["rating"].isnull()]
print(objects)


from copy import deepcopy
df2 = deepcopy(df)
df2.head()

max_cores = max(df["num_cores"].unique())
max_speed = max(df["processor_speed"].unique())

print(max_cores,max_speed)

# Assuming df is your DataFrame
df2['phone_category'] = df2.apply(categorize_phone, axis=1)
df2['camera_category'] = df2.apply(categorize_camera, axis=1)

objects = df2[df["rating"].isnull()]
objects

# Apply np.where based on the phone_category column
df2['rating'] = np.where(df2['rating'].isnull() & (df2['phone_category'] == "flagship"), 90,
                        np.where(df2['rating'].isnull() & (df2['phone_category'] == "midrange"), 65,
                                 np.where(df2['rating'].isnull() & (df2['phone_category'] == "average"), 40, df2['rating'])))

# Apply np.where based on the phone_category column
df2['rating'] = np.where(df2['rating'].isnull() & (df2['phone_category'] == "flagship"), 90,
                        np.where(df2['rating'].isnull() & (df2['phone_category'] == "midrange"), 65,
                                 np.where(df2['rating'].isnull() & (df2['phone_category'] == "average"), 40, df2['rating'])))

df2.head()


    brand_name                        model   price  rating  has_5g  has_nfc  \
14     samsung  Samsung Galaxy S23 Ultra 5G  114990     NaN    True     True   
29     oneplus               OnePlus 11 Pro   69999     NaN    True     True   
37     samsung  Samsung Galaxy S22 Ultra 5G   91999     NaN    True     True   
49     samsung        Samsung Galaxy A74 5G   42999     NaN    True     True   
69        oppo             Oppo Find N Fold   99990     NaN    True     True   
..         ...                          ...     ...     ...     ...      ...   
954     huawei                Huawei Mate X  169000     NaN    True     True   
957       vivo                    Vivo Y55S   13490     NaN   False    False   
963       lava                      Lava X3    6999     NaN   False    False   
972       itel                    itel A23s    4787     NaN   False    False   
974       vivo                Vivo X Fold 2  119990     NaN    True     True   

     has_ir_blaster processor_brand  nu

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,phone_category,camera_category
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,1440 x 3216,3,1.0,android,50.0,16.0,0,NaN,flagship,low
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,1080 x 2412,3,1.0,android,64.0,16.0,1,1024.0,flagship,low
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,1080 x 2408,3,1.0,android,50.0,13.0,1,1024.0,flagship,high
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,1080 x 2400,3,1.0,android,50.0,16.0,1,1024.0,flagship,low
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,1080 x 2412,3,1.0,android,108.0,16.0,0,NaN,flagship,low


In [43]:
final_rec = final_rec.query("brand_name == 'apple'")
final_rec.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
6,apple,Apple iPhone 14,65999,81.0,True,True,False,bionic,6.0,3.22,...,6.1,60,1170 x 2532,2,1.0,ios,12.0,12.0,0,NaN
15,apple,Apple iPhone 13,62999,79.0,True,True,False,bionic,6.0,3.22,...,6.1,60,1170 x 2532,2,1.0,ios,12.0,12.0,0,NaN
27,apple,Apple iPhone 14 Pro Max,129990,76.0,True,True,False,bionic,6.0,NaN,...,6.7,120,1290 x 2796,3,1.0,ios,48.0,12.0,0,NaN
33,apple,Apple iPhone 11,38999,73.0,False,True,False,bionic,6.0,2.65,...,6.1,60,828 x 1792,2,1.0,ios,12.0,12.0,0,NaN
56,apple,Apple iPhone 14 Plus,74999,82.0,True,True,False,bionic,6.0,3.22,...,6.7,60,1284 x 2778,2,1.0,ios,12.0,12.0,0,NaN


In [44]:
final_rec = final_rec.query("price >= 100000")
final_rec.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
27,apple,Apple iPhone 14 Pro Max,129990,76.0,True,True,False,bionic,6.0,NaN,...,6.70,120,1290 x 2796,3,1.0,ios,48.0,12.0,0,NaN
76,apple,Apple iPhone 13 Pro,119900,83.0,True,True,False,bionic,6.0,3.22,...,6.10,120,1170 x 2532,3,1.0,ios,12.0,12.0,0,NaN
100,apple,Apple iPhone 14 Pro,119990,75.0,True,True,False,bionic,6.0,NaN,...,6.10,120,1179 x 2556,3,1.0,ios,48.0,12.0,0,NaN
152,apple,Apple iPhone 15 Pro Max,142990,79.0,True,True,False,bionic,NaN,NaN,...,6.68,120,1284 x 2778,3,1.0,ios,50.0,12.0,0,NaN
161,apple,Apple iPhone 13 Pro Max,129900,84.0,True,True,False,bionic,6.0,3.22,...,6.70,120,1284 x 2778,3,1.0,ios,12.0,12.0,0,NaN


In [45]:
final_rec = final_rec.query("refresh_rate <= 90")
final_rec.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,screen_size,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto
594,apple,Apple iPhone 11 Pro Max,109900,77.0,False,True,False,bionic,6.0,2.65,...,6.5,60,1242 x 2688,3,1.0,ios,12.0,12.0,0,NaN
613,apple,Apple iPhone 12 Pro (512GB),139900,80.0,True,True,False,bionic,6.0,3.10,...,6.1,60,1170 x 2532,3,1.0,ios,12.0,12.0,0,NaN
788,apple,Apple iPhone 14 Plus (512GB),104999,83.0,True,True,False,bionic,6.0,3.22,...,6.7,60,1284 x 2778,2,1.0,ios,12.0,12.0,0,NaN
824,apple,Apple iPhone 12 Pro (256GB),119900,80.0,True,True,False,bionic,6.0,3.10,...,6.1,60,1170 x 2532,3,1.0,ios,12.0,12.0,0,NaN


In [ ]:
pixels = df.query("brand_name == 'google'")


In [ ]:
level5 = ['excellent', 'outstanding','exceptional','remarkable','first-rate','top-notch','exceptionally good',' splendid', 'impressive',"Superlative","Exemplary","Prime","Stellar","Astonishing","Fabulous","Phenomenal","Spectacular","Marvelous","Exceptionally","Incredible","Brilliant","Sublime","Distinguished","Peerless","Unparalleled","Eminent","Unsurpassed","Unbeatable","Epic"]
level1 = adjectives_worst = ["Abysmal","Appalling","Atrocious","Awful","Dreadful","Terrible","Horrendous","Horrible","Deplorable","Disastrous","Disgusting","Execrable","Lousy","Pathetic","Shameful","Unspeakable","Vile","Wretched","Miserable","Catastrophic","Grim","Woeful","Dire","Repulsive","Rotten","Detestable","Unacceptable","Inferior","Foul","Unsatisfactory"]
categories = [] #performance,camera,software,build,display,photo,video,video, storage,speaker

In [15]:
stage1 = [
    'How do you want to get recommendation?',
    'Entering your own phrase',
    'Get your questions asked'
]

exact_questions = [
    '1.) IPhone 2.) Android 3.)Open to both',
    "Do you want a particular smartphone brand? \n1.) No 2.) Yes (Input your brand)",
    'What screen size would you prefer? \n1.) below 6.1" 2.) between 6.1" & 6.6" 3.) above 6.6" 4.) Doesn\'t matter',
    'Do you want your phone to have 5g? \n1.) Yes 2.) No 3.) Any',
    'What is your budget range Example (inside the "") "45000-50000"?'
]
performance_questions = [
    'Do you frequently play games on your smartphone? (y/n)',
    'Do you use a  lot of apps? (y/n)',
    'Do you want to take high quality photos and videos (y/n)',
    'Do you wish to use your smartphone for over 5 years? (y/n)',
    'Do you wish to have all new features in your smartphone? (y/n)'
]
camera_questions = [
    'Do you want to take high quality photos and videos? (y/n)',
    'Do you want a camera that shoot good in night time? (y/n)'
    'Would you love to shoot zoom and potrait shots? (y/n)'
    'Do you want to shoot many videos? (y/n)',
    'Do you want to shoot many photos? (y/n)'
]
software_questions = [
    'Do you want to get new updates for 5+ years? (y/n)'
    "Do you mind useless apps pre-installed in your smartphone? (y/n)",
    'Do you like to have easy software support? (y/n)'
    'Can you distinguish between smooth and non smooth animation? (y/n)'
]
display_questions = [
    'Do you do a lot of scrolling? (y/n)',
    "Do you like watching movies in mobile? (y/n)",
    'Do you consume a lot of media like youtube and more? (y/n)',
    'Do you wish to use phone a lot on sunlight? (y/n)'
]
storage_questions = [
    "Do you take a lot of photos and videos? (y/n)",
    'Do you store a lot of offline medias like movies or other files in smartphones? (y/n)',
    "Do you want your smartphone to be able to store everything you want for years to come? (y/n)",
    'Do you want to transfer from and to your smartphones quickly? (y/n)'
    'Do you download a lot of apps orr games? (y/n)',
]
speaker_questions = [
    "Would you like a good speaker? \n1.) yes (y) 2.) don't mind (n)"
]
battery_questions = [
    "Do you like a bigger battery? (y/n)",
    'Do you charge very rarely? (y/n)'
    "Do you want fast charging? (y/n)",
    'Do you want wireless charging? (y/n)',
]

all_categories = [exact_questions,performance_questions, camera_questions,software_questions,display_questions,storage_questions,speaker_questions]

In [19]:
for prompts in stage1:
    print(prompts)
lst = []

How do you want to get recommendation?
Entering your own phrase
Get your questions asked


In [23]:
lst = []
for category in all_categories:
    for questions in category:
        lst.append(input(questions))


In [46]:
df2.head()

,brand_name,model,price,rating,has_5g,has_nfc,has_ir_blaster,processor_brand,num_cores,processor_speed,...,refresh_rate,resolution,num_rear_cameras,num_front_cameras,os,primary_camera_rear,primary_camera_front,extended_memory_available,extended_upto,phone_category
0,oneplus,OnePlus 11 5G,54999,89.0,True,True,False,snapdragon,8.0,3.2,...,120,1440 x 3216,3,1.0,android,50.0,16.0,0,NaN,flagship
1,oneplus,OnePlus Nord CE 2 Lite 5G,19989,81.0,True,False,False,snapdragon,8.0,2.2,...,120,1080 x 2412,3,1.0,android,64.0,16.0,1,1024.0,flagship
2,samsung,Samsung Galaxy A14 5G,16499,75.0,True,False,False,exynos,8.0,2.4,...,90,1080 x 2408,3,1.0,android,50.0,13.0,1,1024.0,flagship
3,motorola,Motorola Moto G62 5G,14999,81.0,True,False,False,snapdragon,8.0,2.2,...,120,1080 x 2400,3,1.0,android,50.0,16.0,1,1024.0,flagship
4,realme,Realme 10 Pro Plus,24999,82.0,True,False,False,dimensity,8.0,2.6,...,120,1080 x 2412,3,1.0,android,108.0,16.0,0,NaN,flagship


['3',
 'Google',
 '3',
 '1',
 '3000-50000',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y',
 'y']